In [18]:
import mlflow
from autogluon.tabular import TabularPredictor
import pickle
import pandas as pd

In [7]:
predictor = TabularPredictor.load("AutogluonModels/ag-20260131_224624")

In [5]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("autogluon_experiment")

2026/02/12 14:30:32 INFO mlflow.tracking.fluent: Experiment with name 'autogluon_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1770917432517, experiment_id='1', last_update_time=1770917432517, lifecycle_stage='active', name='autogluon_experiment', tags={}>

In [14]:
model = pickle.load(open(predictor.path + "/predictor.pkl", "rb"))

with mlflow.start_run() as run:

    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=model
    )

    model_uri = f"runs:/{run.info.run_id}/model"

    mlflow.register_model(
        model_uri=model_uri,
        name="autogluon_model"
    )


2026/02/12 14:39:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/jef/repos/rating-evaluation/.venv/lib/python3.12/site-packages/mlflow/tracing/provider.py:757: FutureWarning: Passing a Python object as `python_model` causes it to be serialized using CloudPickle, it requires exercising caution as Python object serialization mechanisms may execute arbitrary code during deserialization.Consider using a file path (str or Path) instead. See https://mlflow.org/docs/latest/ml/model/models-from-code/ for details.
  result = f(*args, **kwargs)


🏃 View run delightful-carp-749 at: http://localhost:5000/#/experiments/1/runs/dd1a750b90914377b0562181b9ac662c
🧪 View experiment at: http://localhost:5000/#/experiments/1


MlflowException: `python_model` must be a PythonModel instance, callable object, or path to a script that uses set_model() to set a PythonModel instance or callable object.

In [22]:
import mlflow.pyfunc

class AutoGluonWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        from autogluon.tabular import TabularPredictor
        self.model = TabularPredictor.load(context.artifacts["predictor_path"])

    def predict(self, context, model_input):
        return self.model.predict(model_input)


/home/jef/repos/rating-evaluation/.venv/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [23]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("autogluon_experiment")

with mlflow.start_run() as run:

    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=AutoGluonWrapper(),
        artifacts={"predictor_path": "/home/jef/repos/rating-evaluation/AutogluonModels/ag-20260131_224624"}
    )

    model_uri = f"runs:/{run.info.run_id}/model"

    mlflow.register_model(
        model_uri=model_uri,
        name="autogluon_model"
    )


2026/02/12 14:52:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/jef/repos/rating-evaluation/.venv/lib/python3.12/site-packages/mlflow/tracing/provider.py:757: FutureWarning: Passing a Python object as `python_model` causes it to be serialized using CloudPickle, it requires exercising caution as Python object serialization mechanisms may execute arbitrary code during deserialization.Consider using a file path (str or Path) instead. See https://mlflow.org/docs/latest/ml/model/models-from-code/ for details.
  result = f(*args, **kwargs)
Registered model 'autogluon_model' already exists. Creating a new version of this model...
2026/02/12 14:52:12 WARNING mlflow.tracking._model_registry.fluent: Run with id 24789973fffd4bf8848fcf4445d9d14f has no artifacts at artifact path 'model', registering model based on models:/m-681e22b2641043a99d14626d050e8aff instead
2026/02/12 14:52:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300

🏃 View run rambunctious-tern-24 at: http://127.0.0.1:5000/#/experiments/1/runs/24789973fffd4bf8848fcf4445d9d14f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [25]:
import mlflow.pyfunc

model = mlflow.pyfunc.load_model("models:/autogluon_model/2")
payload = {
    "review": "This product is amazing! I loved it and will buy again."
}
preds = model.predict(pd.DataFrame([payload]))

In [26]:
preds

0    Negative
Name: type, dtype: object